In [1]:
import os
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
from dotenv import load_dotenv
from typing import Literal
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.agents.agent_types import AgentType
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI

from openai import OpenAI


In [2]:
cd ../

d:\Project\Python\Automatic Anlalyz\dagpt


d:\Project\Python\Automatic Anlalyz\dagpt\.venv\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
cd dagpt

[WinError 2] The system cannot find the file specified: 'dagpt'
d:\Project\Python\Automatic Anlalyz\dagpt


d:\Project\Python\Automatic Anlalyz\dagpt\.venv\Lib\site-packages\IPython\core\magics\osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [4]:
# Ensure environment variables are loaded
load_dotenv()

True

In [5]:
import sys
sys.path.append(r'D:\Project\Python\Automatic Anlalyz\dagpt\src')
from src.Logger.base import BaseLogger
from src.models.llms import load_llm


# Check if GEMINI_KEY_API is set
if 'GEMINI_KEY_API' not in os.environ:
	raise KeyError('GEMINI_KEY_API not found in environment variables')

In [6]:
MODEL_NAME = "gemini-1.5-pro"
llm = load_llm(model_name=MODEL_NAME)
llm

genai.GenerativeModel(
    model_name='models/gemini-1.5-pro',
    generation_config={},
    safety_settings={},
    tools=None,
    system_instruction=None,
    cached_content=None
)

In [7]:
response = llm.generate_content("Can you give me more details about crypto?")
print(response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "Crypto, short for cryptocurrency, is a digital or virtual currency designed to work as a medium of exchange. It uses cryptography to secure and verify transactions as well as to control the creation of new units of a particular cryptocurrency.  Here's a breakdown of key aspects:\n\n**Key Concepts:**\n\n* **Decentralization:**  Unlike traditional currencies issued by central banks, cryptocurrencies operate on a decentralized network, typically a blockchain. This means no single entity controls it, making it resistant to censorship and government manipulation.\n* **Blockchain:** A distributed ledger technology that records all transactions across multiple computers. This makes it transparent and virtually tamper-proof.\n* **Cryptography:**  Uses complex mathem

In [13]:
df = pd.read_csv(r".\data\raw\sample_data.csv")


da_agent = create_pandas_dataframe_agent(
        llm=ChatGoogleGenerativeAI(
        # model="gemini-1.5-pro",
        model= MODEL_NAME,
        temperature=0,
        google_api_key=os.getenv("GEMINI_KEY_API")
    ),
    df=df,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    allow_dangerous_code=True,
    verbose=True,
    return_intermediate_steps=True,
)
da_agent

AgentExecutor(verbose=True, agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={'df_head': '|    |        date |    open |    high |     low |   close |\n|---:|------------:|--------:|--------:|--------:|--------:|\n|  0 | 1.63067e+12 | 48734.8 | 48734.8 | 48734.8 | 48734.8 |\n|  1 | 1.63067e+12 | 49820.7 | 49820.7 | 48734.8 | 49807.2 |\n|  2 | 1.63067e+12 | 48753.4 | 48769.6 | 48753.4 | 48769.6 |\n|  3 | 1.63067e+12 | 48769.6 | 48769.6 | 48769.6 | 48769.6 |\n|  4 | 1.63067e+12 | 49314.8 | 49331.2 | 49314.8 | 49331.2 |', 'tools': 'python_repl_ast - A Python shell. Use this to execute python commands. Input should be a valid python command. When using this tool, sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.', 'tool_names': 'python_repl_ast'

In [14]:
query = 'Describe this data'
response = da_agent(query)

C:\Users\DELL\AppData\Local\Temp\ipykernel_2764\4214231101.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = da_agent(query)




> Entering new AgentExecutor chain...
Question: Describe this data
Thought: I can use the `.describe()` method to get a statistical summary of the dataframe.
Action: python_repl_ast
Action Input: 
```python
print(df.describe())
```               date          open           high           low         close
count  4.182000e+03   4182.000000    4182.000000   4182.000000   4182.000000
mean   1.631045e+12  48289.136432   49125.018852  47510.879644  48295.397683
std    2.173291e+08   2561.709599    9686.008587   4613.829317   2568.211632
min    1.630669e+12  34710.000000   40836.420000   9005.000000  34710.000000
25%    1.630857e+12  46075.505000   46168.175000  45693.697500  46075.462500
50%    1.631045e+12  47028.125000   47497.730000  46792.680000  47039.185000
75%    1.631233e+12  50360.245000   50556.690000  50254.577500  50365.215000
max    1.631421e+12  54410.800000  272617.000000  52851.000000  60000.000000
The data appears to be financial time series data, possibly for a cryptocu

In [15]:
query = 'plot scatter plot of "close" and "open"'
response = da_agent(query)



> Entering new AgentExecutor chain...
Thought: I can use the `plot` method of the dataframe to create a scatter plot.

Action: python_repl_ast
Action Input:
```python
df.plot.scatter(x='open', y='close')
```Axes(0.125,0.11;0.775x0.77)Thought: The code executed successfully, generating a scatter plot.  To display the plot, I need to use `plt.show()`.

Action: python_repl_ast
Action Input:
```python
import matplotlib.pyplot as plt
df.plot.scatter(x='open', y='close')
plt.show()
```

<string>:1: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown


Thought: I can use the `plot` method of the dataframe to create a scatter plot.

Action: python_repl_ast
Action Input:
```python
df.plot.scatter(x='open', y='close')
```Axes(0.125,0.11;0.775x0.77)Thought: The code executed successfully, generating a scatter plot.  To display the plot, I need to use `plt.show()`.

Action: python_repl_ast
Action Input:
```python
import matplotlib.pyplot as plt
df.plot.scatter(x='open', y='close')
plt.show()
```

<string>:1: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown


Thought: I can use the `plot` method of the dataframe to create a scatter plot.  I need to import matplotlib to display it.

Action: python_repl_ast
Action Input:
```python
import matplotlib.pyplot as plt
df.plot.scatter(x='open', y='close')
plt.show()
```

<string>:1: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


ResourceExhausted: 429 Resource has been exhausted (e.g. check quota).

In [23]:
import matplotlib
matplotlib.use('Agg')  # Thêm dòng này ở đầu file
import matplotlib.pyplot as plt
import streamlit as st


In [24]:
def plot_scatter(df, x_col, y_col, title=None):
    # Tạo figure mới
    fig = plt.figure(figsize=(10, 6))
    
    # Vẽ scatter plot
    df.plot.scatter(x=x_col, y=y_col)
    
    # Thêm tiêu đề và nhãn
    plt.title(title or f'Scatter Plot of {x_col} vs {y_col}')
    plt.xlabel(x_col)
    plt.ylabel(y_col)
    
    # Hiển thị trong Streamlit
    st.pyplot(fig)
    
    # Clear figure
    plt.close(fig)



2025-01-05 17:16:35.536 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


2025-01-05 17:16:35.952 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-05 17:16:35.954 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [25]:
# Sử dụng
plot_scatter(df, 'open', 'close', 'Price Comparison: Open vs Close')

2025-01-05 17:17:16.123 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-05 17:17:16.340 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-05 17:17:16.340 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [26]:
def plot_scatter(df, x_col, y_col, title=None):
    fig = plt.figure(figsize=(10, 6))
    df.plot.scatter(x=x_col, y=y_col)
    plt.title(title or f'Scatter Plot of {x_col} vs {y_col}')
    
    # Lưu plot vào buffer
    from io import BytesIO
    buf = BytesIO()
    plt.savefig(buf, format='png')
    plt.close(fig)
    
    # Hiển thị ảnh
    st.image(buf)

# Sử dụng
plot_scatter(df, 'open', 'close', 'Price Comparison')

2025-01-05 17:17:55.133 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-05 17:17:55.135 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-05 17:17:55.136 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [29]:
import plotly.express as px

def plot_scatter_plotly(df, x_col, y_col, title=None):
    fig = px.scatter(df, x=x_col, y=y_col, title=title)
    st.plotly_chart(fig)

# Sử dụng
plot_scatter_plotly(df, 'open', 'close', 'Price Comparison')

2025-01-05 20:53:52.896 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-05 20:53:52.897 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-05 20:53:52.899 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-05 20:53:52.901 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

data = {'date': [1630672000000, 1630675600000, 1630679200000, 1630682800000, 1630686400000],
        'open': [48734.8, 49820.7, 48753.4, 48769.6, 49314.8],
        'high': [48734.8, 49820.7, 48769.6, 48769.6, 49331.2],
        'low': [48734.8, 48734.8, 48753.4, 48769.6, 49314.8],
        'close': [48734.8, 49807.2, 48769.6, 48769.6, 49331.2]}
df = pd.DataFrame(data)
df['date'] = pd.to_datetime(df['date'], unit='ms')
plt.plot(df['date'], df['close'])
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.title('Close Price Over Time')
plt.show()